In [1]:
# pip install langchain --upgrade
# Version: 0.0.164

# !pip install pypdf

In [2]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

### Load your data

In [3]:
loader = PyPDFLoader("example pdfs/NDgwOTA5MQ8686_2019_pdf.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [4]:
data = loader.load()

In [5]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 65 document(s) in your data
There are 468 characters in your document


### Chunk your data up into smaller documents

In [6]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [7]:
print (f'Now you have {len(texts)} documents')

Now you have 59 documents


### Create embeddings of your documents to get ready for semantic search

In [8]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

c:\Users\majal\AppData\Local\Programs\Python\Python39\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [9]:
# Check to see if there is an environment variable with you API keys, if not, use what you put below
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'YourAPIKey')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'northamerica-northeast1-gcp') # You may need to switch with your env

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [11]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "crownprincecomp"

In [12]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [13]:
query = "what is this document about"
docs = docsearch.similarity_search(query)


In [14]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

#engine -> db -> waleed 0.97 ? omar 0.8 abdullah 0.8 hamza0.7

with the same ease. At one moment you can use the computer to  prepare a letter 
document and in the next moment you may play music or  print a document.  
Computers have several limitations too. Computer can only perform tasks that 
it has  been programmed to do.


### Query those docs to get your answer back

In [15]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [16]:
llm = OpenAI(temperature=0.4, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [17]:
query="ما تعريف الحزب"
docs = docsearch.similarity_search(query)

In [18]:
response =chain.run(input_documents=docs, question=query)
type(response)

str